In [5]:
#For training concat:

tr_path= "/kaggle/input/nsvd-dataset/train_concat/output_h5.h5"#volume:2.09 GB
tr_path_cap= "/kaggle/input/nsvd-dataset/caption/caption_train.h5"
 
#For validation concat

val_path = "/kaggle/input/nsvd-dataset/val_concat/output_h5.h5" #Volume: 1.5 GB
val_path_cap= "/kaggle/input/nsvd-dataset/caption/caption_val.h5"

test_path = "/kaggle/input/nsvd-dataset/test_concat/output_h5.h5"

test_path_cap= "/kaggle/input/nsvd-dataset/caption/caption_test.h5"

In [4]:
import h5py

def get_dataset_info(file_path):
    with h5py.File(file_path, 'r') as h5_file:
        
        for dataset_name, dataset in h5_file.items():
            print(f"Dataset Name: {dataset_name}")
            print(f"Dataset Shape: {dataset.shape}")


file_path = "/kaggle/input/nsvd-dataset/val_concat/output_h5.h5"
get_dataset_info(file_path)

Dataset Name: answers
Dataset Shape: (2500,)
Dataset Name: captionProgs
Dataset Shape: (250, 66)
Dataset Name: captions
Dataset Shape: (250, 21)
Dataset Name: histories
Dataset Shape: (2500, 200)
Dataset Name: historiesProg
Dataset Shape: (2500, 60)
Dataset Name: questionImgIdx
Dataset Shape: (2500,)
Dataset Name: questionProgs
Dataset Shape: (2500, 6)
Dataset Name: questionRounds
Dataset Shape: (2500,)
Dataset Name: questions
Dataset Shape: (2500, 21)


In [7]:
# !rm -r "/kaggle/working/tr_cap_s.h5"

In [8]:
import h5py
import cupy as cp
import numpy as np

def reduce_dataset(input_file, output_file):
    with h5py.File(input_file, 'r') as h5_input:
        
        with h5py.File(output_file, 'w') as h5_output:
            # Iterate through datasets in the input file
            for dataset_name, dataset in h5_input.items():
                # Transfer data to GPU
                gpu_data = cp.array(dataset)
                
                print("size_in: name {} size {}".format(dataset_name, dataset.shape[0]))
                dataset_size = dataset.shape[0]

                reduced_size = dataset_size // 10
                reduced_gpu_data = gpu_data[:reduced_size]

                # Transfer data back to CPU
                reduced_data = cp.asnumpy(reduced_gpu_data)

                h5_output.create_dataset(dataset_name, data=reduced_data)

input_file_path = '/kaggle/input/nsvd-dataset/caption/caption_test.h5'
output_file_path = '/kaggle/working/cap_test_75000.h5'
reduce_dataset(input_file_path, output_file_path)
print("after reduce size:")
get_dataset_info(output_file_path)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


size_in: name answers size 750000
size_in: name captionProgs size 75000
size_in: name captions size 75000
size_in: name histories size 750000
size_in: name historiesProg size 750000
size_in: name questionImgIdx size 750000
size_in: name questionProgs size 750000
size_in: name questionRounds size 750000
size_in: name questions size 750000
after reduce size:
Dataset Name: answers
Dataset Shape: (75000,)
Dataset Name: captionProgs
Dataset Shape: (7500, 66)
Dataset Name: captions
Dataset Shape: (7500, 21)
Dataset Name: histories
Dataset Shape: (75000, 200)
Dataset Name: historiesProg
Dataset Shape: (75000, 60)
Dataset Name: questionImgIdx
Dataset Shape: (75000,)
Dataset Name: questionProgs
Dataset Shape: (75000, 6)
Dataset Name: questionRounds
Dataset Shape: (75000,)
Dataset Name: questions
Dataset Shape: (75000, 21)
